In [1]:
from pyspark.sql import SparkSession
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from pyspark.sql.functions import pandas_udf
import pandas as pd
import re
from typing import List, Dict
from pyspark.sql.functions import udf, lit, col
from pyspark.sql.types import StringType, ArrayType
from transformers import pipeline
import os

/home/ec2-user/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# df['text'].str.len().sort_values()

In [4]:
df.head()

,text
0,I love sci-fi and am willing to put up with a ...
1,"Worth the entertainment value of a rental, esp..."
2,its a totally average film with a few semi-alr...
3,STAR RATING: ***** Saturday Night **** Friday ...
4,"First off let me say, If you haven't enjoyed a..."


In [2]:
spark = SparkSession.builder \
    .master("spark://10.0.9.154:7077")\
    .appName("S3Access Demo") \
    .config("spark.executor.memory", "1536m")\
    .config("spark.python.worker.memory", "1536m")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/01 11:16:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/01 11:16:40 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [ ]:
spark = SparkSession.builder \
    .master("spark://10.0.9.154:7077") \
    .appName("S3Access") \
    .config("spark.executor.memory", "1536m") \
    .config("spark.python.worker.memory", "1536m") \
    .config("spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2") \
    .config("spark.hadoop.mapreduce.outputcommitter.factory.scheme.s3a", "org.apache.hadoop.fs.s3a.commit.S3ACommitterFactory") \
    .config("spark.hadoop.mapreduce.output.committer.class", "org.apache.hadoop.fs.s3a.commit.S3ACommitterFactory") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk:1.12.262,org.apache.spark:spark-hadoop-cloud_2.12:3.3.1") \
    .getOrCreate()


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("spark://10.0.9.154:7077") \
    .appName("S3Access") \
    .config("spark.executor.memory", "1536m") \
    .config("spark.python.worker.memory", "1536m") \
    .config("spark.hadoop.fs.s3a.bucket.all.committer.magic.enabled", "true") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk:1.12.262,org.apache.spark:spark-hadoop-cloud_2.12:3.3.1") \
    .getOrCreate()


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk added as a dependency
org.apache.spark#spark-hadoop-cloud_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f240b6e2-fabc-4e63-a483-01520d44c880;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found com.amazonaws#aws-java-sdk;1.12.262 in central
	found com.amazonaws#aws-java-sdk-iamrolesanywhere;1.12.262 in central
	found com.amazonaws#aws-java-sdk-core;1.12.262 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	fo

In [3]:
df = spark.read.csv("s3a://comp0239-ucabryo/test-data/20240404145324_imdb.csv", header=True)

24/04/04 14:59:30 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [4]:
df

DataFrame[text: string]

In [8]:
df = spark.createDataFrame(df)

In [4]:
df = df.limit(50)

In [11]:

def clean_text(text: str) -> str:
    text = re.sub(r'http\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

clean_text_udf = udf(clean_text, StringType())

In [12]:
df = df.withColumn("cleaned_text", clean_text_udf(col("text")))

In [26]:
df.show()

+--------------------+--------------------+
|                text|        cleaned_text|
+--------------------+--------------------+
|I love sci-fi and...|I love scifi and ...|
|Worth the enterta...|Worth the enterta...|
|its a totally ave...|its a totally ave...|
|STAR RATING: ****...|STAR RATING Satur...|
|First off let me ...|First off let me ...|
|I had high hopes ...|I had high hopes ...|
|Isaac Florentine ...|Isaac Florentine ...|
|It actually pains...|It actually pains...|
|Technically I'am ...|Technically Iam a...|
|Honestly awful fi...|Honestly awful fi...|
|This flick is a w...|This flick is a w...|
|Blind Date (Colum...|Blind Date Columb...|
|I first watched t...|I first watched t...|
|I saw the Mogul V...|I saw the Mogul V...|
|A group of heirs ...|A group of heirs ...|
|Now, I LOVE Itali...|Now I LOVE Italia...|
|This cheap, grain...|This cheap grainy...|
|I just finished w...|I just finished w...|
|Ben, (Rupert Grin...|Ben Rupert Grint ...|
|Every movie I hav...|Every movi

In [5]:
# classifier = pipeline(task="text-classification", model="distilbert-base-uncased-finetuned-sst-2-english", top_k=None)
# classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', 
#                       truncation=True, top_k = None)
classifier = pipeline(task="text-classification", model="philschmid/tiny-bert-sst2-distilled", truncation=True, top_k=None)

@pandas_udf('string')
def predict_udf(texts: pd.Series) -> pd.Series:
    prediction = [result[0]['label'] for result in classifier(texts.to_list(), batch_size=2)]
    return pd.Series(prediction)

In [5]:
# classifier = pipeline(task="text-classification", model="distilbert-base-uncased-finetuned-sst-2-english", truncation=True, top_k=None)
# classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', 
#                       truncation=True, top_k = None)
# classifier = pipeline(task="text-classification", model="philschmid/tiny-bert-sst2-distilled", truncation=True, top_k=None)
classifier = pipeline(task="text-classification", model="philschmid/tiny-bert-sst2-distilled", truncation=True, top_k=None)

from transformers import AutoTokenizer

# Initialize the tokenizer for your specific model
tokenizer = AutoTokenizer.from_pretrained('bhadresh-savani/distilbert-base-uncased-emotion')

# Access the model's max length attribute
max_length = tokenizer.model_max_length

print(f"The maximum length the model can handle is: {max_length} tokens")

The maximum length the model can handle is: 512 tokens


In [15]:
# classifier(df['text'].to_list())
texts = ["I am not having a great day",\
             "I am having a great day","I am not having a great day"]

classifier(texts)

[[{'label': 'negative', 'score': 0.9900599718093872},
  {'label': 'positive', 'score': 0.009939965792000294}],
 [{'label': 'positive', 'score': 0.9995552897453308},
  {'label': 'negative', 'score': 0.0004446818493306637}],
 [{'label': 'negative', 'score': 0.9900599718093872},
  {'label': 'positive', 'score': 0.009939965792000294}]]

In [6]:
df = df.withColumn("prediction", predict_udf(col("text")))

In [7]:
jobid = "20240323093728"
df = df.withColumn("jobid", lit(jobid))

In [8]:
df.show()

+--------------------+----------+--------------+
|                text|prediction|         jobid|
+--------------------+----------+--------------+
|"I love sci-fi an...|  negative|20240323093728|
|Worth the enterta...|  negative|20240323093728|
|its a totally ave...|  negative|20240323093728|
|STAR RATING: ****...|  negative|20240323093728|
|First off let me ...|  positive|20240323093728|
|I had high hopes ...|  negative|20240323093728|
|Isaac Florentine ...|  positive|20240323093728|
|It actually pains...|  negative|20240323093728|
|Technically I'am ...|  negative|20240323093728|
|"Honestly awful f...|  negative|20240323093728|
|"This flick is a ...|  negative|20240323093728|
|Blind Date (Colum...|  negative|20240323093728|
|"I first watched ...|  negative|20240323093728|
|"I saw the Mogul ...|  negative|20240323093728|
|A group of heirs ...|  negative|20240323093728|
|"Now, I LOVE Ital...|  negative|20240323093728|
|This cheap, grain...|  negative|20240323093728|
|I just finished w..

In [8]:
# data_folder = os.path.join('/home/ec2-user/data/result', str(jobid))
data_folder = f"s3a://comp0239-ucabryo/spark-result/{jobid}/"
df.write.csv(data_folder, mode="overwrite")

In [6]:
df.count()

50

## test results


In [17]:
test = pd.read_csv("s3://comp0239-ucabryo/result/20240323093728.csv")

In [18]:
test

,text,jobid,prediction
0,"you owe me $3.99!""",2.024032e+13,joy
1,joy,NaN,NaN
2,I just finished watching this movie and am dis...,2.024032e+13,anger
3,"Ben, (Rupert Grint), is a deeply unhappy adole...",2.024032e+13,joy
4,kill off a main character. And then bring him...,2.024032e+13,sadness
5,"Technically I'am a Van Damme Fan, or I was. th...",2.024032e+13,sadness
6,"Honestly awful film, bad editing, awful lighti...",2.024032e+13,sadness


In [9]:
import s3fs
import pandas as pd

# Initialize an s3fs object
fs = s3fs.S3FileSystem(anon=False)  # Use anon=False if you're using AWS credentials

job_id = "20240323093728"
bucket_name = "comp0239-ucabryo"
prefix = f"spark-result/{job_id}/"

# Construct the S3 'directory' to list
s3_directory = f"{bucket_name}/{prefix}"

# List all files in the directory that start with 'part-'
# s3fs handles paths in S3 like file paths, so you can use glob patterns.
data_list = fs.glob(f"{s3_directory}part-*")

In [10]:
data_list

['comp0239-ucabryo/spark-result/20240323093728/part-00000-80cd3c5d-4fcf-47d2-9d1b-350f3875a3ca-c000.csv']

In [15]:
# job_id = "20240323093728"
# # data_folder = f'/home/ec2-user/data/result/{job_id}'
# data_folder = f"s3://comp0239-ucabryo/spark-result/{job_id}/"
# # Correctly list files starting with 'part-'
# data_list = [f for f in os.listdir(data_folder) if f.startswith("part-")]

dfs = []  # Use a list to collect DataFrames

for file in data_list:
    file_path = f"s3://{file}"
    df_part = pd.read_csv(file_path,
                          names=["text", "prediction", "jobid"],  # Column names if there's no header
                            header=None,  # Use if the first row is not a header
                            escapechar="\\",  # Helps if your text includes quotes
                            quotechar='"')
    dfs.append(df_part)  # Append each part to the list


In [16]:
dfs[0]

,text,prediction,jobid
0,"Set during the Hungarian Revolution of 1956, t...",negative,20240323093728
1,I have seen several Yul Brynner films--yet thi...,positive,20240323093728
2,"Yul Brynner is Major Surov, a singing, dancing...",negative,20240323093728
3,"""I really enjoy this movie. The first time it ...",positive,20240323093728
4,"a great movie, with a rather unclear political...",positive,20240323093728
5,"""As a big Dostoyevsky fan, I had always been d...",positive,20240323093728
6,"""""""The Journey"""" is a very good film. Produced...",positive,20240323093728
7,Yul Brynner was a symbol of villein in the tin...,positive,20240323093728
8,"""Most people will consider that Yul Brynner's ...",positive,20240323093728
9,There have been many excellent comments about ...,positive,20240323093728


In [17]:

res = pd.concat(dfs, axis=0)  # Concatenate all at once

# Ensure s3fs is installed to use "s3://" schema
res.to_csv(f"s3://comp0239-ucabryo/result/{job_id}.csv", index=False)


24/04/03 16:39:40 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
24/04/03 16:39:40 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:291)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:981)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:165)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:263)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:170)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce

In [24]:
res

,text,prediction,jobid
0,Every movie I have PPV'd because Leonard Malti...,joy,20240323093728
1,Low budget horror movie. If you don't raise yo...,joy,20240323093728
0,I just finished watching this movie and am dis...,anger,20240323093728
1,"Ben, (Rupert Grint), is a deeply unhappy adole...",joy,20240323093728
0,This flick is a waste of time.I expect from an...,sadness,20240323093728
1,"Blind Date (Columbia Pictures, 1934), was a de...",sadness,20240323093728
0,"Worth the entertainment value of a rental, esp...",joy,20240323093728
1,its a totally average film with a few semi-alr...,sadness,20240323093728
